In [ ]:
import netCDF4 as nc
import pandas as pd

In [ ]:
dataset = nc.Dataset('path-to/NOAAGlobalTemp_v5.0.0_gridded_s188001_e202212_c20230108T133308.nc', 'r')
dataset

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    Conventions: CF-1.6, ACDD-1.3
    title: NOAA Merged Land Ocean Global Surface Temperature Analysis (NOAAGlobalTemp)
    summary: NOAAGlobalTemp is a merged land-ocean surface temperature analysis, formerly abbreviated as MLOST. The SST dataset is the Extended Reconstructed Sea Surface Temperature (ERSST) and the land surface air temperature dataset is from the Global Historical Climatology Network Monthly (GHCN-M) database.
    institution: DOC/NOAA/NESDIS/National Centers for Environmental Information(NCEI)
    id:  gov.noaa.ncdc:C00934 
    naming_authority:  gov.noaa.ncei 
    project:  NOAAGlobalTemp 
    platform: Ships, moored buoys, surface drifting buoys, Argo floats, and weather stations
    instrument: Conventional thermometers
    history: Version 5.0.0, blending ERSST V5 and GHCN-M V4.
    comment: Merged land ocean surface temperature anomalies.
    product_version: 5.0.0
    source:

In [ ]:
dataset2 = nc.Dataset('path-to/precip.mon.total.v7.nc', 'r')
#dataset2

In [ ]:
variable_names = dataset.variables.keys()
print(variable_names)

dict_keys(['time', 'lat', 'lon', 'z', 'anom'])


In [ ]:
lat = dataset.variables ['lat']
lat

<class 'netCDF4._netCDF4.Variable'>
float32 lat(lat)
    long_name: Latitude
    standard_name: latitude
    coverage_content_type: coordinate
    units: degrees_north
    grids: Uniform grid from -87.5 to 87.5 by 5
    valid_min: -87.5
    valid_max: 87.5
    axis: Y
    _CoordinateAxisType: Lat
    coordinate_defines: center
unlimited dimensions: 
current shape = (36,)
filling on, default _FillValue of 9.969209968386869e+36 used

In [ ]:
temp = dataset.variables ['anom']

In [ ]:
temp

<class 'netCDF4._netCDF4.Variable'>
float32 anom(time, z, lat, lon)
    long_name: Global Temperature Anomalies
    standard_name: surface_temperature_anomaly
    coverage_content_type: physicalMeasurement
    units: degrees C
    missing_value: -999.9
    _FillValue: -999.9
    add_offset: 0.0
    scale_factor: 1.0
    valid_min: -40.0
    valid_max: 40.0
    coordinates: time z lat lon
unlimited dimensions: 
current shape = (1716, 1, 36, 72)
filling on

In [ ]:
temp_data = dataset.variables['anom'][:]

In [ ]:
lat_min, lat_max = 1.318, 20.633
lon_min, lon_max = 9.053, 31.113

In [ ]:
import numpy as np
lat = dataset.variables['lat'][:]
lon = dataset.variables['lon'][:]
#lat_indices = np.where(lat >= lat_min) & (lat <= lat_max)[0]
#lon_indices = np.where(lon >= lon_min) & (lon <= lon_max)[0]
lat_indices = np.searchsorted(lat, [lat_min, lat_max])
lon_indices = np.searchsorted(lon, [lon_min, lon_max])

In [ ]:
df = pd.DataFrame(lon_indices)
df1 = pd.DataFrame(lat_indices)
df

,0
0,2
1,6


In [ ]:
subset_temp_data = temp_data[:, :, lat_indices[0]:lat_indices[1], lon_indices[0]:lon_indices[1]]
subset_temp_data

masked_array(
  data=[[[[--, --, --, --],
          [--, --, --, --],
          [--, --, --, --],
          [--, --, --, --]]],


        [[[--, --, --, --],
          [--, --, --, --],
          [--, --, --, --],
          [--, --, --, --]]],


        [[[--, --, --, --],
          [--, --, --, --],
          [--, --, --, --],
          [--, --, --, --]]],


        ...,


        [[[1.115722894668579, 1.3334460258483887, 1.733852505683899,
           1.736942172050476],
          [0.5634891390800476, 0.8149952292442322, 1.1685633659362793,
           1.23863685131073],
          [-0.13441035151481628, -0.21748760342597961,
           -0.022795557975769043, -0.07569509744644165],
          [-0.21568381786346436, -0.46628880500793457,
           -0.3644026219844818, 0.12995927035808563]]],


        [[[1.065653920173645, 1.1346665620803833, 0.9089114665985107,
           1.0080486536026],
          [1.0774524211883545, 1.1668460369110107, 0.32432568073272705,
           0.2417601197957

In [ ]:
subset_temp_data
data = subset_temp_data.data
mask = subset_temp_data.mask

In [ ]:
mask_df = pd.DataFrame(mask.reshape(-1, 4))
data_df = pd.DataFrame(data.reshape(-1, 4))
data_df = data_df.replace(-999.900024, pd.NA)
print(data_df)

               0           1           2           3
0    -999.900024 -999.900024 -999.900024 -999.900024
1    -999.900024 -999.900024 -999.900024 -999.900024
2    -999.900024 -999.900024 -999.900024 -999.900024
3    -999.900024 -999.900024 -999.900024 -999.900024
4    -999.900024 -999.900024 -999.900024 -999.900024
...          ...         ...         ...         ...
6859   -0.370762    0.039638    0.013788    0.514658
6860    1.017878    0.699228    0.834526    0.664569
6861    1.287798    1.480371    0.574885    0.485178
6862    0.557464    0.764912    0.965372    0.557712
6863    1.238358    1.307567    0.734556    1.106015

[6864 rows x 4 columns]


In [ ]:
DF = pd.concat([data_df, mask_df], axis = 1)
DF

,0,1,2,3,0,1,2,3
0,-999.900024,-999.900024,-999.900024,-999.900024,True,True,True,True
1,-999.900024,-999.900024,-999.900024,-999.900024,True,True,True,True
2,-999.900024,-999.900024,-999.900024,-999.900024,True,True,True,True
3,-999.900024,-999.900024,-999.900024,-999.900024,True,True,True,True
4,-999.900024,-999.900024,-999.900024,-999.900024,True,True,True,True
...,...,...,...,...,...,...,...,...
6859,-0.370762,0.039638,0.013788,0.514658,False,False,False,False
6860,1.017878,0.699228,0.834526,0.664569,False,False,False,False
6861,1.287798,1.480371,0.574885,0.485178,False,False,False,False
6862,0.557464,0.764912,0.965372,0.557712,False,False,False,False
